<a href="https://colab.research.google.com/github/rohit9650/ML/blob/master/DeepDream/DeepDream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://cdn.photographylife.com/wp-content/uploads/2016/06/Mass.jpg

--2019-02-09 20:57:54--  https://cdn.photographylife.com/wp-content/uploads/2016/06/Mass.jpg
Resolving cdn.photographylife.com (cdn.photographylife.com)... 68.70.205.3, 68.70.205.4, 68.70.205.1, ...
Connecting to cdn.photographylife.com (cdn.photographylife.com)|68.70.205.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 361459 (353K) [image/jpeg]
Saving to: ‘Mass.jpg’

Mass.jpg            100%[===================>] 352.99K   844KB/s    in 0.4s    

2019-02-09 20:57:55 (844 KB/s) - ‘Mass.jpg’ saved [361459/361459]

--2019-02-09 20:57:54--  https://cdn.photographylife.com/wp-content/uploads/2016/06/Mass.jpg
Resolving cdn.photographylife.com (cdn.photographylife.com)... 68.70.205.3, 68.70.205.4, 68.70.205.1, ...
Connecting to cdn.photographylife.com (cdn.photographylife.com)|68.70.205.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 361459 (353K) [image/jpeg]
Saving to: ‘Mass.jpg’

Mass.jpg            100%[===================>] 352.99K

In [0]:
# Loading the pretrained Inception V3 model

from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet', 
                                 include_top=False)

In [0]:
# Setting up the DeepDream configuration
'''
Dictionary mapping layer names to a coefficient quantifying
how much the layer’s activation contributes to the loss
you’ll seek to maximize. Note that the layer names are
hardcoded in the built-in Inception V3 application.
'''

layers_contributions = {'mixed2': 0.2,
                        'mixed3': 3.,
                        'mixed4': 2.,
                        'mixed5':1.5}

In [0]:
# Defining the loss to be maximized

layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)

for layer_name in layers_contributions:
  coeff = layers_contributions[layer_name]
  activation = layer_dict[layer_name].output
  
  scaling = K.prod(K.cast(K.shape(activation), 'float32'))
  loss = loss + (coeff *
                 K.sum(K.square(activation[:, 2: -2, 2: -2, :]))/ scaling)

In [0]:
# Gradient-ascent process

dream = model.input

grads = K.gradients(loss, dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
  outs = fetch_loss_and_grads([x])
  
  return outs[0], outs[1]

def gradient_ascent(x, iterations, step, max_loss=None):
  '''This function runs gradient ascent for a number of iterations.'''
  for i in range(iterations):
    loss_value, grad_values = eval_loss_and_grads(x)
    if max_loss is not None and loss_value > max_loss:
      break
    print('...Loss value at ', i, ' : ', loss_value)
    x += step * grad_values
  return x

In [0]:
# Auxiliary functions

import scipy
from keras.preprocessing import image

def resize_img(img, size):
  img = np.copy(img)
  factors = (1, 
             float(size[0]) / img.shape[1],
             float(size[1]) / img.shape[2],
             1)
  
  return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
  pil_img = deprocess_image(np.copy(img))
  scipy.misc.imsave(fname, pil_img)

def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  
  img = np.expand_dims(img, axis=0)
  img = inception_v3.preprocess_input(img)
  
  return img

def deprocess_image(x):
  if K.image_data_format() == 'channels_first':
    x = x.reshape((3, x.shape[2], x.shape[3]))
    x = x.transpose((1, 2, 0))
  else:
    x = x.reshape((x.shape[1], x.shape[2], 3))
  
  x /= 2
  x += 0.5
  x *= 255
  x = np.clip(x, 0, 255).astype('uint8')
  
  return x

In [19]:
# Running gradient ascent over different successive scales

import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = 'Mass.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]

for i in range(1, num_octave+1):
  shape = tuple([int(dim / (octave_scale ** i)) 
                 for dim in original_shape])
  successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
  print('Processing image shape : ', shape)
  img = resize_img(img, shape)
  img = gradient_ascent(img,
                        iterations=iterations,
                        step=step,
                        max_loss=max_loss)
  
  upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
  same_size_original = resize_img(original_img, shape)
  lost_details = same_size_original - upscaled_shrunk_original_img
  
  img += lost_details
  shrunk_original_img = resize_img(original_img, shape)
  
  save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')

Processing image shape :  (497, 746)
...Loss value at  0  :  1.488431
...Loss value at  1  :  1.9446623
...Loss value at  2  :  2.6635947
...Loss value at  3  :  3.5067208
...Loss value at  4  :  4.369861
...Loss value at  5  :  5.2346787
...Loss value at  6  :  6.050723
...Loss value at  7  :  6.8767757
...Loss value at  8  :  7.6460466
...Loss value at  9  :  8.406577
...Loss value at  10  :  9.137813
...Loss value at  11  :  9.85883


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  app.launch_new_instance()


Processing image shape :  (696, 1044)
...Loss value at  0  :  3.480019
...Loss value at  1  :  4.9060893
...Loss value at  2  :  6.118775
...Loss value at  3  :  7.1910787
...Loss value at  4  :  8.185787
...Loss value at  5  :  9.131754
Processing image shape :  (975, 1462)
...Loss value at  0  :  3.3428652
...Loss value at  1  :  4.6267705
...Loss value at  2  :  5.782117
...Loss value at  3  :  6.8565397
...Loss value at  4  :  7.8900533
...Loss value at  5  :  8.875455
...Loss value at  6  :  9.823857
Processing image shape :  (1365, 2048)
...Loss value at  0  :  3.5813951
...Loss value at  1  :  4.932485
...Loss value at  2  :  6.1044493
...Loss value at  3  :  7.223404
...Loss value at  4  :  8.324343
...Loss value at  5  :  9.393098
